# INTELIGENCIA ARTIFICIAL (1INF24)
Dr. Edwin Villanueva Talavera (ervillanueva@pucp.edu.pe)

## Búsqueda local para solucionar tableros de sudoku
Este notebook implementa búsqueda local para encontrar soluciones a tableros de sudoku. Se les está disponibilizando la implementación del algoritmo Simulating Annealing y Hill Climbing.

### Funciones utilitarias para manejar el tablero del sudoku
Estas son funciones utilitarias para trabajar con tableros de sudoku.

In [1]:
import sys
import time
import numpy as np
from random import shuffle, random, sample, randint
from copy import deepcopy
from math import exp

# Funcion para cargar el tablero del sudoku de un archivo
def readBoardFromFile(filename):
    fd = open(filename, "r+")
    puzzle = eval(fd.readline())
    board = []
    for row in puzzle:
        for col in row:
            board.append(col)
    return np.array(board)  # El tablero es un vector con las filas concatenadas

# Devuelve los indices de los elementos de la columna i del tablero
def get_column_indices(i):
    indices = [i + 9 * j for j in range(9)]
    return indices

# Devuelve los indices de los elementos de la fila i del tablero
def get_row_indices(i):
    indices = [j + 9 * i for j in range(9)]
    return indices

# Devuelve los indices de los elementos del bloque k del tablero. initialEntries son los indices de las posiciones inmutables (numeros iniciales)
# si ignore_originals=True entonces solo devuelve los indices de las posiciones que no son las posiciones inmutables
def get_block_indices(k, initialEntries, ignore_originals=False):
    row_offset = (k // 3) * 3
    col_offset = (k % 3) * 3
    indices = [col_offset + (j % 3) + 9 * (row_offset + (j // 3)) for j in range(9)]
    if ignore_originals:
        indices = [x for x in indices if x not in initialEntries]
    return indices

# Completa aleatoriamente las pociciones vacias (que no son initialEntries).
# Garantiza que en cada bloque se generen 9 numeros diferentes
def randomAssign(board, initialEntries):
    for num in range(9):
        block_indices = get_block_indices(num, initialEntries)
        block = board[block_indices]
        zero_indices=[ind for i, ind in enumerate(block_indices) if block[i] == 0]
        to_fill = [i for i in range(1, 10) if i not in block]
        shuffle(to_fill)
        for ind, value in zip(zero_indices, to_fill):
            board[ind] = value

# Evalua un tablero. Devuelve la cantidad de numeros diferentes en todas las filas y columnas
# un tablero solucion tiene un score de 162 (81 + 81)
def score_board(board):
    score = 0
    for row in range(9):  # Por cada fila obtiene la cantidad de numeros diferentes
        score+= len(set(board[get_row_indices(row)]))
    for col in range(9):  # Por cada columna obtiene la cantidad de numeros diferentes
        score += len(set(board[get_column_indices(col)]))
    return score

# Genera un tablero nuevo. Se escoje aleatoriamente un bloque y se intercambia dos valores (no dados en el tablero original)
def make_neighborBoard(board, initialEntries):
    new_board = deepcopy(board)
    block = randint(0, 8)  # Escoje un bloque aleatoriamente
    num_in_block = len(get_block_indices(block, initialEntries, ignore_originals=True))  # Cantidad de posiciones que se puede alterar en el bloque
    random_squares = sample(range(num_in_block), 2)  # Escoje dos posiciones aleatorias del bloque para intercambiar valores
    square1, square2 = [get_block_indices(block, initialEntries, ignore_originals=True) [ind] for ind in random_squares]
    new_board[square1], new_board[square2] = new_board[square2], new_board[square1]  # Intercambia los valores de las posiciones
    return new_board

# Imprime un tablero en pantalla
def showBoard(board):
    def checkZero(s):
        if s != 0: return str(s)
        if s == 0: return "0"
    results = np.array([board[get_row_indices(j)] for j in range(9)])
    s=""
    for i, row in enumerate(results):
        if i % 3 == 0:
            s += "-" * 25 + '\n'
        s += "| " + " | ".join([" ".join(checkZero(s) for s in list(row)[3 * (k - 1) : 3 * k]) for k in range(1, 4)]) + " |\n"
    s += "-" * 25 + ''
    print(s)


### Implementación del Algoritmo Hill Climbing para resolver tableros de sudoku

In [2]:
def get_neighborhood(board, initialEntries, number_of_neighbors=100):
    """Finds neighboring boards using make_neighborBoard() method."""
    neighborhood = []
    for i in range(number_of_neighbors):
        neighborhood.append(make_neighborBoard(board, initialEntries))
    return neighborhood

def hc_solver(board, number_of_neighbors=100, maxIter=1000):
    """ Hill Climbing solver. From the initial board, keep choosing the neighbor board with highest score,
        stopping when no neighbor is better.
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...
        number_of_neighbors: Number of neighbors in the neighborhood
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print('Hill Climbing intentará resolver el siguiente tablero sudoku:')
    showBoard(board)
    initialEntries = np.arange(81)[board > 0]  # Las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes

    current_board = board
    current_score = score_board(board)
    t = 0
    while (t < maxIter):  # Bucle principal
        if (t % 100 == 0):
            print('Iteration {},\tCurrent score = {}'.format(t, current_score))

        neighborhood = get_neighborhood(current_board, initialEntries, number_of_neighbors)  # Obtiene tableros vecinos
        if not neighborhood:
            break

        neighborhood_scores = []
        for i in range(len(neighborhood)):  # Evaluate each neighbor board
            neighborhood_scores.append( score_board(neighborhood[i]) )
        index_best_neighbor = np.argmax(neighborhood_scores)  # The index of the best neighbor

        if neighborhood_scores[index_best_neighbor] >= current_score:
            current_score = neighborhood_scores[index_best_neighbor]
            current_board = deepcopy(neighborhood[index_best_neighbor])
        t += 1

    end_time = time.time()
    print("Best board found in {} seconds ({} ierations) with {}/{} points:".format(end_time-start_time,t,current_score,162))
    showBoard(current_board)


### Implementación del Algoritmo Simulating Annealing para resolver tableros de sudoku

In [3]:
def sa_solver(board, T0=0.5, DR=.999999, maxIter=100000):
    """ Simulating annealing solver.
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...
        T0: Initial temperature
        DR: The decay rate of the schedule function: Ti = T0*(DR)^i (Ti is the temperature at iteration i).
            For efficiecy the schedule function is implemented as: Ti = T(i-1)*DR
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Simulated Annealing intentará resolver el siguiente tablero sudoku:')
    showBoard(board)

    initialEntries = np.arange(81)[board > 0]  # Las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes
    #showBoard(board)
    best_board = deepcopy(board)
    current_score = score_board(board)
    best_score = current_score
    T = T0  # El valor inicial de la temperatura
    t = 0
    while (t < maxIter):
        try:
            if (t % 10000 == 0):
                print('Iteration {},\tTemperaure = {},\tBest score = {},\tCurrent score = {}'.format(t, T, best_score, current_score))

            neighborBoard = make_neighborBoard(board, initialEntries)  # Obtiene un tablero vecino
            neighborBoardScore = score_board(neighborBoard)  # Evalua tablero vecino
            delta = float(neighborBoardScore - current_score)

            if (delta > 0):  # Si el tablero vecino generado es mejor que el actual, se acepta
                board = neighborBoard
                current_score = neighborBoardScore

            elif (exp((delta / T)) > random()):  # Si el tablero generado es peor, se acepta con prob exp((delta/T))
                board = neighborBoard
                current_score = neighborBoardScore


            if (current_score > best_score):  # Si el tablero actual es mejor que el actual mejor
                best_board = deepcopy(board)
                best_score = score_board(best_board)

            if neighborBoardScore == 162:  # 162 es un tablero solucion
                best_board = neighborBoard
                break

            T = DR * T  # La nueva temperatura decae con factor de decaimiento, es una forma eficiente del schedule Ti = T0 * (DR) ^ i
            t += 1
        except:
            print("Numerical error occurred. It's a random algorithm so try again.")
    end_time = time.time()
    if best_score == 162:
        print ("Solution found in {} seconds (iterations={}):".format(end_time - start_time, t))
    else:
        print("Couldn't solve! ({}/{} points). This is the best solution found (iterations={}):".format(best_score,162,t))

    showBoard(best_board)


## Experimentación con los algoritmos de búsqueda

Llama a Hill Climbing para resolver el tablero 'puzzleA.txt' con number_of_neighbors = 100 y número de iteraciones maxIter = 1000

In [4]:
board = readBoardFromFile('puzzleA.txt')
hc_solver(board, number_of_neighbors=100, maxIter=1000)

Hill Climbing intentará resolver el siguiente tablero sudoku:
-------------------------
| 0 0 1 | 4 0 0 | 3 8 0 |
| 0 0 0 | 6 3 0 | 1 0 0 |
| 0 3 8 | 0 0 0 | 0 7 0 |
-------------------------
| 0 0 3 | 0 0 1 | 0 9 0 |
| 0 6 0 | 0 7 3 | 0 0 0 |
| 0 0 0 | 0 0 0 | 4 0 0 |
-------------------------
| 0 0 0 | 0 0 0 | 0 4 0 |
| 2 1 0 | 0 6 0 | 0 0 0 |
| 4 0 0 | 9 0 0 | 7 0 0 |
-------------------------
Iteration 0,	Current score = 114
Iteration 100,	Current score = 160
Iteration 200,	Current score = 160
Iteration 300,	Current score = 160
Iteration 400,	Current score = 160
Iteration 500,	Current score = 160
Iteration 600,	Current score = 160
Iteration 700,	Current score = 160
Iteration 800,	Current score = 160
Iteration 900,	Current score = 160
Best board found in 19.57283353805542 seconds (1000 ierations) with 160/162 points:
-------------------------
| 7 2 1 | 4 8 5 | 3 8 9 |
| 5 4 9 | 6 3 7 | 1 2 6 |
| 6 3 8 | 1 2 9 | 5 7 4 |
-------------------------
| 8 5 3 | 2 4 1 | 6 9 7 |
| 9 6 4 | 5 

Llama a Simulating Annealing para resolver el tablero 'puzzleA.txt' con Temperatura inicial T0 = 0.5, Factor de decaimiento de temperatura DR = 0.99999 y Máximo número de iteraciones maxIter = 100000

In [6]:
board = readBoardFromFile('puzzleA.txt')
sa_solver(board, T0=1, DR=.999999, maxIter=100000)

Simulated Annealing intentará resolver el siguiente tablero sudoku:
-------------------------
| 0 0 1 | 4 0 0 | 3 8 0 |
| 0 0 0 | 6 3 0 | 1 0 0 |
| 0 3 8 | 0 0 0 | 0 7 0 |
-------------------------
| 0 0 3 | 0 0 1 | 0 9 0 |
| 0 6 0 | 0 7 3 | 0 0 0 |
| 0 0 0 | 0 0 0 | 4 0 0 |
-------------------------
| 0 0 0 | 0 0 0 | 0 4 0 |
| 2 1 0 | 0 6 0 | 0 0 0 |
| 4 0 0 | 9 0 0 | 7 0 0 |
-------------------------
Iteration 0,	Temperaure = 1,	Best score = 112,	Current score = 112
Iteration 10000,	Temperaure = 0.9900498287986292,	Best score = 145,	Current score = 131
Iteration 20000,	Temperaure = 0.9801986635041949,	Best score = 146,	Current score = 132
Iteration 30000,	Temperaure = 0.9704455189909783,	Best score = 146,	Current score = 133
Iteration 40000,	Temperaure = 0.9607894199354179,	Best score = 147,	Current score = 134
Iteration 50000,	Temperaure = 0.9512294007185967,	Best score = 147,	Current score = 135
Iteration 60000,	Temperaure = 0.9417645053296724,	Best score = 151,	Current score = 134